# Load Orders Data from Silver Layer

This section loads the `orders_silver` table from the silver layer in Azure Databricks. The table contains order details such as `order_id`, `customer_id`, `product_id`, `order_date`, `quantity`, and `total_amount`.

In [0]:
from pyspark.sql.functions import *; 
from pyspark.sql.window import Window; 

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@azuresadatalake.dfs.core.windows.net/orders")

In [0]:
display(df)


In [0]:
df.printSchema()

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))

df.display()

In [0]:
from pyspark.sql.functions import month; 

df = df.withColumn("year", year(col("order_date")))

In [0]:
df.display()

In [0]:
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()


In [0]:
class window_data:
    def dense_rank(self, df):
        return (df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount")))))
    def rank(self, df):
        return (df.withColumn("flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount")))))
    def row_number(self, df):
        return (df.withColumn("flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount")))))



In [0]:
obj = window_data()

In [0]:
df_result = obj.dense_rank(df)
df_result.display()


In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@azuresadatalake.dfs.core.windows.net/orders")

In [0]:
%sql
create table if not exists databricks_catalog.silver.orders_silver using delta 
location "abfss://silver@azuresadatalake.dfs.core.windows.net/orders"